In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import datetime as dt
import time
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.stats import poisson
import scipy.stats as stats
%matplotlib inline

from google.oauth2.service_account import Credentials
import google_auth_httplib2
import pygsheets
import googleapiclient
from googleapiclient  import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from gspread_formatting.dataframe import format_with_dataframe, BasicFormatter
from gspread_formatting import Color
import logging

from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gslides
from gslides import (Frame,Presentation,Spreadsheet,Table,Series,Chart)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

#pip install google-api-core==1.32.0
#pip install --upgrade firebase-admin

In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
from firebase_admin import auth

#pip install google-api-core==1.32.0
#pip install protobuf==3.15.5
#pip install google-cloud-firestore==2.3.4


###>>>>>>>>>>>>>>>>>>>>>>>>>> Firestore credentials
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate("C:/Users/leemn/OneDrive/Documents/Service Account/accountkey_firebase.json")
    firebase_admin.initialize_app(cred)
db = firestore.client() 

In [3]:
data_trips = pd.read_excel('C:/Users/leemn/Downloads/LoopDriverDistance20221122_final.xlsx')
data_trips.head()

,client_id,order_id,trip_id,order_no,date,TipDividedBy2,actualKM,predictedKM,distanceKM,user_type,...,pickup_completed,delivery_started,delivery_completed,driver,name,order_type_id,vendor,mall,city,onTime
0,B0l6eGBH0UB7wOhWzZ9J,dZ98RYwMCdQyY4nYAdkB,Brnk9aHrN1lWTVDrdVKY,3111843.0,2022-11-18,5.0,0.0,8.90,9.0,13.0,...,2022-11-18 08:43:49,2022-11-18 08:43:53,2022-11-18 09:05:12,Kande Matonda,Kande,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes
1,B0l6eGBH0UB7wOhWzZ9J,aw9GJite668Q3hPNZQ73,BA7iqR3iHXhFA0Q6Nr2w,3111906.0,2022-11-18,5.0,0.0,4.53,5.0,13.0,...,2022-11-18 09:07:01,2022-11-18 09:07:15,2022-11-18 09:32:02,Ntobeko Mhlongo,Ntobeko,2.0,NaN,Gateway Mall,Durban,yes
2,B0l6eGBH0UB7wOhWzZ9J,QJNCbxNTPHoHG87jwHdV,mtHI1QUoSaghkP9XZPui,3112123.0,2022-11-18,5.0,0.0,3.34,4.0,13.0,...,2022-11-18 08:45:46,2022-11-18 08:48:38,2022-11-18 09:02:19,Jackson Mzoza,Jackson,2.0,NaN,Westville Mall,Durban,yes
3,B0l6eGBH0UB7wOhWzZ9J,JWi2crb4S4CjF3tjBVYc,KwtFodIFeeKyeTGFlI0i,3112161.0,2022-11-18,5.0,0.0,5.06,6.0,13.0,...,2022-11-18 08:41:47,2022-11-18 08:42:48,2022-11-18 08:53:59,Tharcisse Muamba,Tharcisse,2.0,NaN,Woodbridge Square Mall,Johannesburg,yes
4,B0l6eGBH0UB7wOhWzZ9J,ETzfdJ6Acpr6VPuxE18X,aSMJ0Q5DgRZU7qaCXgqL,3112208.0,2022-11-18,5.0,0.0,5.51,6.0,13.0,...,2022-11-18 13:18:06,2022-11-18 13:18:36,2022-11-18 13:34:06,Premlal Ramraj,Premlal,2.0,NaN,Gateway Mall,Durban,yes


In [6]:
batch = db.batch()

# Update for one
docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document('OC5kvIDRk7JDFTSScahY')
batch.update(docs, {u'wallet.cost': 40})
batch.update(docs, {u'distance': "4.04"})
batch.commit()

[update_time {
   seconds: 1668774309
   nanos: 597894000
 },
 update_time {
   seconds: 1668774309
   nanos: 597894000
 }]

In [15]:
data_trips = data_trips[~data_trips['trip_id'].isnull()]
                        
#df[~df['B'].isnull()]                    

In [6]:
script_path = os.path.abspath('Update Firestore records.ipynb')
script_dir = os.path.split(script_path)[0]
Error_path = r"Errors"
abs_error_path = os.path.join(script_dir, Error_path)
print(abs_error_path)

C:\Users\leemn\Errors


In [7]:
import traceback

In [5]:
df1, df2, df3, df4, df5, df6 = np.array_split(data_trips[data_trips['client_id']!='VCm2MFBrnAaMLQaImiJf'], 6)

In [6]:
# first update
batch_6 = db.batch()

for trip_id,driver_earnings,predictedKM in tqdm(zip(df1.trip_id, df1.driver_earnings, df1.predictedKM)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_6.update(docs, {u'wallet.cost': driver_earnings})
        batch_6.update(docs, {u'distance': str(predictedKM)})
        batch_6.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

270it [01:34,  2.87it/s]


In [7]:
batch_7 = db.batch()

for trip_id,driver_earnings,predictedKM in tqdm(zip(df2.trip_id, df2.driver_earnings, df2.predictedKM)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_7.update(docs, {u'wallet.cost': driver_earnings})
        batch_7.update(docs, {u'distance': str(predictedKM)})
        batch_7.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

270it [01:18,  3.46it/s]


In [8]:
batch_8 = db.batch()

for trip_id,driver_earnings,predictedKM in tqdm(zip(df3.trip_id, df3.driver_earnings, df3.predictedKM)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_8.update(docs, {u'wallet.cost': driver_earnings})
        batch_8.update(docs, {u'distance': str(predictedKM)})
        batch_8.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

270it [01:18,  3.46it/s]


In [9]:
batch_9 = db.batch()

for trip_id,driver_earnings,predictedKM in tqdm(zip(df4.trip_id, df4.driver_earnings, df4.predictedKM)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_9.update(docs, {u'wallet.cost': driver_earnings})
        batch_9.update(docs, {u'distance': str(predictedKM)})
        batch_9.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

270it [01:19,  3.41it/s]


In [10]:
batch_10 = db.batch()

for trip_id,driver_earnings,predictedKM in tqdm(zip(df5.trip_id, df5.driver_earnings, df5.predictedKM)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_10.update(docs, {u'wallet.cost': driver_earnings})
        batch_10.update(docs, {u'distance': str(predictedKM)})
        batch_10.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

270it [01:18,  3.45it/s]


In [21]:
data_trips = pd.read_excel('C:/Users/leemn/Downloads/correct2.xlsx',sheet_name='correct')
data_trips.head()

,trip_id,order_id,order_no,predictedKM,tip_amount,driver_earnings
0,0EtY3VTl5aPdxLlRyFNY,6R39g82vWnt3Z1u8TLha,4015129.0,7.380,10.0,68.0
1,0ZcPl2Y3lzS94jdEKDRf,GEaRkC8Nm7s8kYr1xh8x,4014643.0,9.328,10.0,80.0
2,12aRMZCCEjLoom6FGgVg,G4CmC3HhFGR0HJzIBLaS,4015114.0,5.141,10.0,56.0
3,14SY1rjlkH5el6zxUdx7,Vdv3eF5YBZPchpAlxxOm,4015413.0,5.632,0.0,46.0
4,17EsN0UU8usV6J59kLHG,a2T4mihFMfehxk2ZYOx6,4015081.0,6.017,0.0,52.0


In [22]:
batch_11 = db.batch()

for trip_id,driver_earnings,predictedKM,tip_amount in tqdm(zip(data_trips.trip_id, data_trips.driver_earnings, 
                                                               data_trips.predictedKM, data_trips.tip_amount)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch_11.update(docs, {u'wallet.cost': driver_earnings})
        batch_11.update(docs, {u'wallet.tip_amount': tip_amount})
        batch_11.update(docs, {u'distance': str(predictedKM)})
        batch_11.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

242it [01:09,  3.48it/s]


## Update Wallet Amount to 0 for these orders -  Woolies Dash

In [3]:
data_trips = pd.read_excel('C:/Users/leemn/Downloads/set_to_zero.xlsx')
data_trips.head()

,order_id,order_no,trip_id,driver_earnings
0,BL7nRTcP7SpPpwHBbXAy,3993882.0,DiPAOoLaUkO5NCQNchv2,0.0
1,QkJ5ZR5Fpja9J9zlLCSn,3994593.0,p1DaOxiiqjh9lerXUcf4,0.0
2,5XG2iNqCUZKI9BNry5qI,3994875.0,w6cei4n8RLF1WWLvGl2o,0.0
3,spOvpsCQA7o0EG6pERcU,3994899.0,g0rSsCLk7uT9aPwynHYU,0.0
4,6ak5PEQD5eskzJw9EjoR,3994778.0,WcXcwK80bB1m9yu4JJu5,0.0


In [4]:
batch = db.batch()

for trip_id, driver_earnings in tqdm(zip(data_trips.trip_id, data_trips.driver_earnings)):
    try:
        docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips').document(trip_id)
        #print(trip_id)
        #print(driver_earnings)
        #print(predictedKM)
        batch.update(docs, {u'wallet.cost': driver_earnings})
        #batch.update(docs, {u'distance': str(predictedKM)})
        batch.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()    

170it [00:47,  3.55it/s]


### Update User permissions scopes firestore

In [47]:
# reference to the clients document
clients_ref = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J')

# reference to the users collection under the clients document
users_ref = clients_ref.collection('users')

# iterate over the query results and update the permissions.scope field
for user_doc in users_ref.stream():
    # get the email address of the current user
    email = user_doc.get('email')
    print(f"User email: {email}")

User email: rea@onecart.co.za
User email: vuyo@onecart.co.za
User email: noma@onecart.co.za
User email: tshepiso@onecart.co.za
User email: theo@onecart.co.za
User email: clinton@onecart.co.za
User email: jillian@onecart.co.za
User email: louis@onecart.co.za
User email: ronald@onecart.co.za
User email: brandon@onecart.co.za
User email: bongani.d@onecart.co.za
User email: nkululekom@onecart.co.za
User email: constance@onecart.co.za
User email: melita@onecart.co.za
User email: nonhlanhla@onecart.co.za
User email: pearl@onecart.co.za
User email: tshepo.d@onecart.co.za
User email: irene@onecart.co.za
User email: kaylie@onecart.co.za
User email: lebogang.x@onecart.co.za
User email: nobuhle@onecart.co.za
User email: masego@onecart.co.za
User email: moe@onecart.co.za
User email: nhlanhla@onecart.co.za
User email: sipho@onecart.co.za
User email: mbalenhle@onecart.co.za
User email: terese@onecart.co.za
User email: francisco@onecart.co.za
User email: brian@onecart.co.za
User email: nkululeko@onec

In [50]:
# reference to the clients document
clients_ref = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J')

# reference to the users collection under the clients document
users_ref = clients_ref.collection('users')

# iterate over the query results and update the permissions.scope field
for user_doc in users_ref.stream():
    # exclude the user with email 'do-not-delete@loop.co.za'
    if user_doc.get('email') == 'do-not-delete@loop.co.za':
        continue
    
    # reference to the user's client-users document
    client_users_ref = user_doc.get('user_client_ref')
    
    # get the DocumentReference object from the DocumentSnapshot object
    client_users_doc_ref = client_users_ref.get().reference
    
    # update the permissions.scope field
    client_users_doc_ref.update({'permissions.scopes': ['orders:none', 'import:none']})

## Deleting a field

In [52]:
# reference to the clients document
clients_ref = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J')

# reference to the users collection under the clients document
users_ref = clients_ref.collection('users')

# iterate over the query results and delete the 'scopes' field
for user_doc in users_ref.stream():
    # exclude the user with email 'do-not-delete@loop.co.za'
    if user_doc.get('email') == 'do-not-delete@loop.co.za':
        continue

    # reference to the user's client-users document
    client_users_ref = user_doc.get('user_client_ref')
    
    # get the DocumentReference object from the DocumentSnapshot object
    client_users_doc_ref = client_users_ref.get().reference
    
    # delete the 'scopes' field
    client_users_doc_ref.update({'scopes': firestore.DELETE_FIELD})

### Update vehicle type field

In [4]:
credentials = Credentials.from_service_account_file('C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json')

In [5]:
df = pd.read_excel(r'C:/Users/leemn/Downloads/df_final_drivers.xlsx',
                   sheet_name='df_final_drivers')
df.head()

,vehicle_type,driver_email,client_id,driver_id,driver_name,driver_mobile,employee_code
0,Bike,mashupejohn88@gmail.com,B0l6eGBH0UB7wOhWzZ9J,hkyVBvE1j6t7caoMJROr,John Mashupe,+27671852511,W-1098
1,Bike,mashupejohn88@gmail.com,B0l6eGBH0UB7wOhWzZ9J,hkyVBvE1j6t7caoMJROr,John Mashupe,+27671852511,W-1098
2,Car,ziyandapeter@gmail.com,B0l6eGBH0UB7wOhWzZ9J,dvkOs4nkilHFDf53bhEo,Ziyanda Peter,+27839984488,W-1029
3,Bike,zipporahb@gmail.com,B0l6eGBH0UB7wOhWzZ9J,EchdeE9RAybtClKvgQVU,Zipporah Bakali,+27694470884,WooliesDash167
4,Bike,zestpiousm@gmail.com,B0l6eGBH0UB7wOhWzZ9J,svOEjAnBUvddsdxfw1mf,Zest Makonde,+27789748665,WooliesDash149


In [8]:
batch_7 = db.batch()

for driver_id, vehicle_type in tqdm(zip(df.driver_id, df.vehicle_type)):
    try:
        docs = db.collection('drivers').document(driver_id)
        #print(driver_id)
        #print(vehicle_type)
        batch_7.update(docs, {u'vehicle_type': vehicle_type})
        batch_7.commit()
    except Exception as e:
        log = open(abs_error_path+'\\'+str(trip_id)+".txt", "a")
        log.write(':\n')
        traceback.print_exc(file=log)      
        log.write('\n')
        log.close()  

639it [02:56,  3.63it/s]
